In [ ]:
# %load Main_script2.py
"""
Created on Thu May  5 16:17:53 2022

@author: salih
"""

from cmath import sqrt

from matplotlib.pyplot import axis
from sklearn.cluster import KMeans
from elbow import calculateKWithElbow

import numpy as np
import pandas as pd
import scipy.spatial.distance as metric
import math
import sklearn.datasets as datasets
import time
import warnings
warnings.filterwarnings("ignore")


iris = datasets.load_iris()


def calcSSE(data, centroids):
    sum = 0
    for i in data:
        distance = math.inf
        for k in centroids:
            if euc(i, k) < distance:
                distance = euc(i, k)
        sum += distance ** 2

    return sum / len(data)


def euc(A, B):
    # Call to scipy with vector parameters

    return metric.euclidean(A, B)


def rand_cent(ds, k):
    # Number of columns in dataset
    n = np.shape(ds)[1]

    # The centroids
    centroids = np.mat(np.zeros((k, n)))

    # Create random centroids
    for j in range(n):

        min_j = min(ds[:, j])
        range_j = float(max(ds[:, j]) - min_j)
        centroids[:, j] = min_j + range_j * np.random.rand(k, 1)

    # Return centroids as numpy array
    return centroids


def kmeans(ds, k, cent_method):
    global timer_start
    global timer_end
    global total_timer_end
    return_object = {}
    global cents
    global sse
    global iters
    cents = []

    if cent_method == "random":
        timer_start = time.perf_counter()
        cents = rand_cent(ds, k)
        timer_end = time.perf_counter()
    elif cent_method == "naive":
        timer_start = time.perf_counter()
        cents = naive_sharding(ds, k)
        timer_end = time.perf_counter()
    elif cent_method == "mean":
        timer_start = time.perf_counter()
        cents = mean_sharding(ds, k)
        timer_end = time.perf_counter()
    elif cent_method == "median":
        timer_start = time.perf_counter()
        cents = median_sharding(ds, k)
        timer_end = time.perf_counter()
    elif cent_method == "minmax":
        timer_start = time.perf_counter()
        cents = minmaxsharding(ds, k)
        timer_end = time.perf_counter()
    km = KMeans(n_clusters=k, init=cents).fit(ds)
    total_timer_end = time.perf_counter()
    iters = km.n_iter_
    cents = km.cluster_centers_
    sse = km.inertia_
    return_object['cents'] = cents
    return_object['time'] = timer_end - timer_start
    return_object['total-time'] = total_timer_end - timer_start
    return_object['sse'] = sse / len(ds)
    return_object['type'] = cent_method
    return_object['iter'] = iters
    return return_object


def _get_mean(sums, step):
    return sums/step


def naive_sharding(ds, k):
    n = np.shape(ds)[1]

    m = np.shape(ds)[0]

    centroids = np.mat(np.zeros((k, n)))

    composite = np.sum(ds, axis=1)
    composite = np.reshape(composite, (len(ds), 1))

    ds = np.append(composite, ds, axis=1)

    ds.sort(axis=0)
    step = math.floor(m/k)

    vfunc = np.vectorize(_get_mean)

    for j in range(k):
        if j == k-1:
            centroids[j:] = vfunc(np.sum(ds[j*step:, 1:], axis=0), step)
        else:
            centroids[j:] = vfunc(
                np.sum(ds[j*step:(j+1)*step, 1:], axis=0), step)

    return centroids


def mean_sharding(ds, k):
    n = np.shape(ds)[1]

    m = np.shape(ds)[0]

    centroids = np.mat(np.zeros((k, n)))

    composite = np.mean(ds, axis=1)
    composite = np.reshape(composite, (len(ds), 1))

    ds = np.append(composite, ds, axis=1)

    # ds = ds[ds[:, 0].argsort(kind="mergesort")]
    ds.sort(axis=0)

    step = math.floor(m/k)

    vfunc = np.vectorize(_get_mean)

    for j in range(k):
        if j == k-1:
            centroids[j:] = vfunc(np.sum(ds[j*step:, 1:], axis=0), step)
        else:
            centroids[j:] = vfunc(
                np.sum(ds[j*step:(j+1)*step, 1:], axis=0), step)

    return centroids


def median_sharding(ds, k):
    n = np.shape(ds)[1]

    m = np.shape(ds)[0]

    centroids = np.mat(np.zeros((k, n)))

    composite = np.median(ds, axis=1)
    composite = np.reshape(composite, (len(ds), 1))

    ds = np.append(composite, ds, axis=1)

    # ds = ds[ds[:, 0].argsort()]
    ds.sort(axis=0)

    step = math.floor(m/k)

    vfunc = np.vectorize(_get_mean)

    for j in range(k):
        if j == k-1:
            centroids[j:] = vfunc(np.sum(ds[j*step:, 1:], axis=0), step)
        else:
            centroids[j:] = vfunc(
                np.sum(ds[j*step:(j+1)*step, 1:], axis=0), step)

    return centroids


def minmaxsharding(ds, k):

    n = np.shape(ds)[1]

    centroids = np.mat(np.zeros((k, n)))

    composite = np.sum(ds, axis=1)

    composite = np.reshape(composite, (len(ds), 1))

    ds = np.append(composite, ds, axis=1)
    # print(ds)

    # ds = ds[ds[:, 0].argsort()]
    ds.sort(axis=0)

    # print(ds)
    ds_range = np.max(ds[:, 0])-np.min(ds[:, 0])

    #threshold = math.ceil(ds_range/k)
    threshold=ds_range/k
    prev_arr = split_arr(ds, threshold, 0)

    for j in range(k):
        # print(prev_arr[1])
        centroids[j, :] = np.sum(
            prev_arr[1][:, 1:], axis=0)/np.shape(prev_arr[1])[0]
        # print(centroids)

        prev_arr = split_arr(ds[prev_arr[0]:, :], threshold, prev_arr[0])
        # print("done")

    return centroids


def split_arr(ds, threshold, j):
    if np.size(ds) == 0:
        return None
    min_val = ds[0, 0]

    k = 0
    for i in range(len(ds)):
        if ds[k, 0]-min_val <= threshold:
            # print(k)
            k += 1
        else:
            break

    return [j+k, ds[0:k, :]]


def printResult(datas):
    print("{:<20} {:<20} {:<20} {:<20} {:<20}".format(
        'Type', 'Time', "SSE", "Total Time", "Iter"))
    print('-----------------------------------------------------------------------------------')
    for d in datas:
        print("{:<20} {:<20} {:<20} {:<20} {:<20}".format(
            d['type'], d['time'], d['sse'], d['total-time'], d['iter']))


#df = pd.DataFrame(iris.data) iris.data nparrayi, bunu df ye çevirip tekrar nparray e çeviriyormuşuz.
#df = df.to_numpy()
df=iris.data
printResult([kmeans(df, 3, 'random'), kmeans(
    df, 3, 'minmax'), kmeans(df, 3, 'median'), kmeans(df, 3, 'mean'), kmeans(df, 3, 'naive')])


Type                 Time                 SSE                  Total Time           Iter                
-----------------------------------------------------------------------------------
random               0.00022320006974041462 0.5257044388398486   0.003077800036408007 11                  
minmax               0.00034879997838288546 0.5256762761743068   0.0025853998959064484 4                   
median               0.0004181999247521162 0.5257044388398486   0.0024910999927669764 3                   
mean                 0.0003108000382781029 0.5257044388398486   0.002436300041154027 3                   
naive                0.00030200008768588305 0.5257044388398486   0.0026119000976905227 3                   


In [ ]:
num=len(iris.data[0])
for i in range(num):
    m=max(iris.data[:,i])
    #print(m)
    iris.data[:,i]=iris.data[:,i]/m
    


In [ ]:
printResult([kmeans(df, 3, 'random'), kmeans(
    df, 3, 'minmax'), kmeans(df, 3, 'median'), kmeans(df, 3, 'mean'), kmeans(df, 3, 'naive')])

Type                 Time                 SSE                  Total Time           Iter                
-----------------------------------------------------------------------------------
random               0.00031779997516423464 0.018835544027599058 0.04991199995856732  9                   
minmax               0.00039570010267198086 0.018835544027599058 0.0026858000783249736 6                   
median               0.00040250003803521395 0.018835544027599058 0.002573500038124621 3                   
mean                 0.00032190000638365746 0.018835544027599058 0.0023722000187262893 3                   
naive                0.00030310009606182575 0.018835544027599058 0.002418900025077164 3                   
